In [2]:
import sys 
sys.path.insert(1, "src/")
from split_model import split_model
from auto_build import bit_build, estimate_report, fits_kv260
from multiprocessing import Process
from qonnx.core.modelwrapper import ModelWrapper

def get_processes():
    model = ModelWrapper("{}/{}".format(args.model_dir,args.model_file))
    model_dir = args.model_dir
    
    processes = []
    op_type = args.split_op_type
    print(model_dir)
    splits = []
    for n in model.graph.node:
        if n.op_type == op_type:
            splits.append(n.output)

    for sn in splits[0::args.jumps_per_op]:
        split_node = sn[0] 
        print("--"*20,split_node, "--"*20)
        # Split model to get a new model
        model_file = split_model(split_node,op_type,model_dir)
        print(model_file)

        # Estimate reports
        final_output_dir = "build-KV260/estimation/{}/{}".format(op_type,split_node)        
        # final_output_dir = "build-{}/{}/{}".format("KV260",op_type,split_node)        
        folding_config_file = "folding_config/auto_build_folding.json"
        estimate_report(model_file,final_output_dir,folding_config_file)

        # Check if estimate fits on KV260, If fits make a bit file
        resource_report = "{}/report/estimate_layer_resources.json".format(final_output_dir)
        if fits_kv260(resource_report):
            final_output_dir = "build-{}/{}/{}".format("KV260",op_type, split_node)
            processes.append(Process(target=bit_build,args=(model_file,final_output_dir,folding_config_file,split_node,)))
        else:
            print("--"*20,"Does not fit", "--"*20)
            print("\t"*20, split_node)
#             ! rm -rf {model_file}
            break
    return processes

if __name__ == "__main__":
    pids = get_processes()
    for p in pids:
        p.start()
    for p in pids:
        p.join()

---------------------------------------- Mul_0_out0 ----------------------------------------
---------------------------------------- Mul_5_out0 ----------------------------------------
---------------------------------------- Mul_10_out0 ----------------------------------------
---------------------------------------- Mul_15_out0 ----------------------------------------
---------------------------------------- Mul_20_out0 ----------------------------------------
---------------------------------------- Mul_25_out0 ----------------------------------------
---------------------------------------- Mul_30_out0 ----------------------------------------
---------------------------------------- Mul_35_out0 ----------------------------------------
---------------------------------------- Mul_40_out0 ----------------------------------------
---------------------------------------- Mul_45_out0 ----------------------------------------
---------------------------------------- Mul_50_out0 ---------

In [2]:
print("--"*20,"Does not fit", "--"*20)

----------------------------------------
